# Running a Multi Objective Optimization Directly in Python

This notebook demonstrates how to:

Write a basic Wrapper in Python and launch a multi objective optimization from Python.
If you wanted to launch it from command line, you would do a similar thing of defining the Wrapper, and then put in your configuration file the information about where the wrapper is, and use [BOA's](../index.rst) CLI tools. See [Running an Experiment from Command Line (Python Wrapper)](example_py_run.rst) for more information.

In [1]:
import pathlib

import yaml
import shutil

import boa
from boa.scripts.moo import Wrapper

In [2]:
# Remove old runs to have a clean slate for this example
old_runs = pathlib.Path().resolve().glob("moo_run*")
for path in old_runs:
    shutil.rmtree(path, ignore_errors=True)

## Loading the MOO Config File

In [3]:
config_path = pathlib.Path().resolve().parent.parent / "boa/scripts/moo.yaml"

Here we can see what the configuration file looks like

In [4]:
with open(config_path, 'r') as f:
    file_contents = f.read()
    print (file_contents)

# MultiObjective Optimization config
optimization_options:
    objective_options:
        objective_thresholds:
            - branin >= -18.0
            - currin >= -6.0
        objectives:
            - name: branin
              lower_is_better: False
              noise_sd: 0
            - name: currin
              lower_is_better: False
              noise_sd: 0

    experiment:
        name: "moo_run"
    trials: 50

parameters:
    x0:
        type: range
        bounds: [0, 1]
        value_type: float
    x1:
        type: range
        bounds: [0, 1]
        value_type: float


In [5]:
config = boa.load_jsonlike(config_path)

## The Wrapper

We define our wrapper in wrapper.py and use a synthetic function that stands in for any black box model call

In [6]:
with open(Wrapper.path(), 'r') as f:
    file_contents = f.read()
    print (file_contents)

from pathlib import Path

import numpy as np
import torch

from boa.controller import Controller
from boa.metrics.synthetic_funcs import get_synth_func
from boa.utils import torch_device
from boa.wrappers.base_wrapper import BaseWrapper

tkwargs = {
    "device": torch_device(),
}
Problem = get_synth_func("BraninCurrin")

problem = Problem(negate=True).to(**tkwargs)


class Wrapper(BaseWrapper):
    def run_model(self, trial) -> None:
        pass

    def set_trial_status(self, trial) -> None:
        trial.mark_completed()

    def fetch_trial_data(self, trial, metric_properties, metric_name, *args, **kwargs):
        evaluation = problem(torch.tensor([trial.arm.parameters["x0"], trial.arm.parameters["x1"]]))
        a = float(evaluation[0])
        b = float(evaluation[1])
        return {
            "branin": a,
            "currin": b
        }


def main():
    config_path = Path(__file__).resolve().parent / "moo.yaml"
    wrapper = Wrapper(config_path=config_path)
    controlle

## The Setup

In [7]:
controller = boa.Controller(config_path=config_path, wrapper=Wrapper)

controller.initialize_scheduler()

[INFO 07-10 13:12:16] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x0', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x1', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 07-10 13:12:16] ax.modelbridge.dispatch_utils: Using Models.MOO since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 07-10 13:12:16] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=2 num_trials=None use_batch_trials=False
[INFO 07-10 13:12:16] ax.modelbridge.dispatch_utils: calculated num_initialization_trials=5
[INFO 07-10 13:12:16] ax.modelbridge.dispatch_utils: num_completed_initialization_trials=0 num_remaining_initialization_trials=5
[INFO 07-10 13:12:16] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation s

(Scheduler(experiment=Experiment(moo_run), generation_strategy=GenerationStrategy(name='Sobol+MOO', steps=[Sobol for 5 trials, MOO for subsequent trials]), options=SchedulerOptions(max_pending_trials=10, trial_type=<TrialType.TRIAL: 0>, batch_size=None, total_trials=None, tolerated_trial_failure_rate=0.5, min_failed_trials_for_failure_rate_check=5, log_filepath=None, logging_level=20, ttl_seconds_for_trials=None, init_seconds_between_polls=1, min_seconds_before_poll=1.0, seconds_between_polls_backoff_factor=1.5, timeout_hours=None, run_trials_in_batches=False, debug_log_run_metadata=False, early_stopping_strategy=None, global_stopping_strategy=None, suppress_storage_errors_after_retries=False)),
 <boa.scripts.moo.Wrapper at 0x145c73190>)

## Start our Experiment

The Controller will save our scheduler to JSON after it completes the run so we can reload it at a later time for analysis or to resume our experiment

In [8]:
scheduler = controller.run()

[INFO 2023-07-10 13:12:16,490 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: /Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/moo_run_20230710T131216
Start Time: 20230710T131216
Version: 0.8.3.dev1+g0636f51.d20230411

##############################################

[INFO 07-10 13:12:16] Scheduler: Running trials [0]...
[INFO 07-10 13:12:17] Scheduler: Running trials [1]...
[INFO 07-10 13:12:18] Scheduler: Running trials [2]...
[INFO 07-10 13:12:19] Scheduler: Running trials [3]...
[INFO 07-10 13:12:20] Scheduler: Running trials [4]...
[INFO 07-10 13:12:21] Scheduler: Generated all trials that can be generated currently. Model requires more data to generate more trials.
[INFO 07-10 13:12:21] Scheduler: Retrieved COMPLETED trials: 0 - 4.
[INFO 07-10 13:12:21] Scheduler: Fetching data for trials: 0 - 4.
[ERROR 2023-07-10 13:12:21,676 MainProcess] boa: Object <boa.scripts.moo.Wrapper object at 0x145c73190> pas

## Get the Best Trial

`best_fitted_trials` uses the data to do a fitting from all trials and with the noise levels you provided (or if no noise levels was provided, it assumed an unknown level of noise and inferred the noise level from the trial runs)

In [9]:
trial = scheduler.best_fitted_trials()
trial

{8: {'params': {'x0': 0.0, 'x1': 1.0},
  'means': {'branin': -17.506304078803886, 'currin': -1.1800505789085367},
  'cov_matrix': {'branin': {'branin': 0.00023219490947858334, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 4.246944294570932e-06}}},
 14: {'params': {'x0': 0.06722595104412638, 'x1': 1.0},
  'means': {'branin': -4.008813794965391, 'currin': -3.6613878669425453},
  'cov_matrix': {'branin': {'branin': 0.0002696194851788826, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 5.207734999339682e-06}}},
 15: {'params': {'x0': 0.03180193896164709, 'x1': 1.0},
  'means': {'branin': -8.89988441563846, 'currin': -2.4505460577327822},
  'cov_matrix': {'branin': {'branin': 7.153857500977046e-05, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 1.1097568335905147e-06}}},
 16: {'params': {'x0': 0.11623283268391146, 'x1': 0.8282747946292564},
  'means': {'branin': -0.478322414465131, 'currin': -5.503003487462207},
  'cov_matrix': {'branin': {'branin': 0.000276655942402

if you need the exact points of the best trial, maybe because you need the trial number of the best trial to plot results, or for any other reason, `best_raw_trails` does not do any fitting

In [10]:
trial = scheduler.best_raw_trials()
trial

{8: {'params': {'x0': 0.0, 'x1': 1.0},
  'means': {'branin': -17.508296966552734, 'currin': -1.180408000946045},
  'cov_matrix': {'branin': {'branin': 0.0, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 0.0}}},
 14: {'params': {'x0': 0.06722595104412638, 'x1': 1.0},
  'means': {'branin': -4.00965690612793, 'currin': -3.6614270210266113},
  'cov_matrix': {'branin': {'branin': 0.0, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 0.0}}},
 15: {'params': {'x0': 0.03180193896164709, 'x1': 1.0},
  'means': {'branin': -8.899821281433105, 'currin': -2.450594663619995},
  'cov_matrix': {'branin': {'branin': 0.0, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 0.0}}},
 16: {'params': {'x0': 0.11623283268391146, 'x1': 0.8282747946292564},
  'means': {'branin': -0.47779369354248047, 'currin': -5.503173351287842},
  'cov_matrix': {'branin': {'branin': 0.0, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 0.0}}},
 17: {'params': {'x0': 0.015048417956364869, 'x1': 1.0},
  

## Output All Trials

In [11]:
boa.scheduler_to_df(scheduler)

,trial_index,arm_name,trial_status,generation_method,branin,currin,is_feasible,x0,x1
0,0,0_0,COMPLETED,Sobol,-104.829926,-13.321523,False,0.157186,0.063027
1,1,1_0,COMPLETED,Sobol,-67.450264,-6.262702,False,0.795510,0.548820
2,2,2_0,COMPLETED,Sobol,-44.699284,-11.003670,False,0.162904,0.291876
3,3,3_0,COMPLETED,Sobol,-10.633623,-8.563358,False,0.895887,0.280258
4,4,4_0,COMPLETED,Sobol,-2.309752,-10.189129,False,0.511377,0.240122
5,5,5_0,COMPLETED,MOO,-191.902283,-4.252816,False,0.797650,0.959299
6,6,6_0,COMPLETED,MOO,-31.095140,-5.988050,False,0.234506,0.876196
7,7,7_0,COMPLETED,MOO,-121.471985,-4.946413,False,0.389611,1.000000
8,8,8_0,COMPLETED,MOO,-17.508297,-1.180408,True,0.000000,1.000000
9,9,9_0,COMPLETED,MOO,-52.435886,-6.313510,False,0.398786,0.710204
